<a href="https://colab.research.google.com/github/HariharanPalanisamyUAE/EmailSpamAIModule/blob/main/Emailspam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Email Spam Finder
#Developer: Hariharan
#Github Repos:https://github.com/HariharanPalanisamyUAE/EmailSpamAIModule

# Import necessary libraries
# pandas is used for data manipulation and analysis

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as feature_extraction
import sklearn.svm as svm
# accuracy_score and classification_report are used to evaluate the model's performance
from sklearn.metrics import accuracy_score, classification_report
# Pipeline is used to chain multiple steps (feature extraction and model) into one
from sklearn.pipeline import Pipeline
# TfidfVectorizer converts text data into numerical features using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# load kaggle huggingface
!pip install datasets
from datasets import load_dataset
from sklearn.naive_bayes import MultinomialNB
import joblib



**1.Load Dataset**

In [12]:
# Load the Enron Spam dataset from Hugging Face
# This dataset contains emails labeled as 'spam' or 'ham' (not spam)
# We use 'SetFit/enron_spam' which is a processed version of the Enron spam emails
dataset = load_dataset('SetFit/enron_spam')
df = pd.DataFrame(dataset['train'])
df.head()

Repo card metadata block was not found. Setting CardData to empty.


,message_id,text,label,label_text,subject,message,date
0,33214,any software just for 15 $ - 99 $ understandin...,1,spam,any software just for 15 $ - 99 $,understanding oem software\nlead me not into t...,2005-06-18
1,11929,perspective on ferc regulatory action client c...,0,ham,perspective on ferc regulatory action client c...,"19 th , 2 : 00 pm edt\nperspective on ferc reg...",2001-06-19
2,19784,wanted to try ci 4 lis but thought it was way ...,1,spam,wanted to try ci 4 lis but thought it was way ...,viagra at $ 1 . 12 per dose\nready to boost yo...,2004-09-11
3,2209,"enron / hpl actuals for december 11 , 2000 tec...",0,ham,"enron / hpl actuals for december 11 , 2000",teco tap 30 . 000 / enron ; 120 . 000 / hpl ga...,2000-12-12
4,15880,looking for cheap high - quality software ? ro...,1,spam,looking for cheap high - quality software ? ro...,"water past also , burn , course . gave country...",2005-02-13


**2. Training**

# Convert the training split to a pandas DataFrame for easier manipulation
# The dataset has columns like 'message' (email text) and 'label' (0 for ham, 1 for spam)




In [18]:
X = df['message']
y = df['label']

# Split the data into training and testing sets
# test_size=0.2 means 20% for testing, random_state=42 ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



28558    please do not hesitate to contact robert johns...
28567    enrononline started to see message delivery sl...
5393     ( see attached file : nomform . wk 4 ) ( see a...
20290    hey ,\nlast week , mason and me were talking a...
4678     structure & technology report\nmay 10 th , 200...
                               ...                        
29802    royal replicas mens and womens replica rolex w...
5390     group ,\nthe plan has slightly changed for sub...
860      margaret ,\nthanks for reminding me this issue...
15795    hi ,\nregalis , also known as superviagra or c...
23654    searching for best adult datlng slte ?\nclick ...
Name: message, Length: 25372, dtype: object


3.pipeline

pipeline first vectorizes the text and then applies the classifier. this streamlines the process of feature extraction and model training

In [21]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Step 1: Convert text to TF-IDF features
    ('classifier', MultinomialNB())  # Step 2: Apply Multinomial Naive Bayes classifier
])

# Fit the pipeline to the training data
# This trains both the vectorizer and the model on the training set
pipeline.fit(X_train, y_train)

# Make predictions on the test set
# This uses the trained pipeline to predict labels for the test emails
y_pred = pipeline.predict(X_test)

# Calculate the accuracy of the model
# accuracy_score compares predicted labels to actual labels
accuracy = accuracy_score(y_test, y_pred)

# Calculate the noise level (interpreted as error rate: 1 - accuracy)
# This represents the proportion of misclassifications
noise_level = 1 - accuracy

# Generate a classification report with precision, recall, f1-score, etc.
# This provides detailed metrics on the model's performance
report = classification_report(y_test, y_pred)

# Print the results
# This displays the accuracy, noise level, and full report
print(f"Accuracy: {accuracy}")
print(f"Noise Level (Error Rate): {noise_level}")
print("Classification Report:\n", report)

# Example of using the model as an 'agent' to classify a new email
# This demonstrates how to use the trained model for inference on new data
new_email = ["Hey, this is a test email. Are you free for a meeting?"]
prediction = pipeline.predict(new_email)

# Interpret the prediction
# Assuming 1 means spam, 0 means ham (adjust based on dataset labels)
if prediction[0] == 1:
    print("This email is classified as spam.")
else:
    print("This email is classified as ham (not spam).")


# Save the model to a file (this creates 'spam_model.pkl' in your current directory)
joblib.dump(pipeline, 'hari_spam_model.pkl')
print("Model saved as 'hari_spam_model.pkl'")

Accuracy: 0.9854981084489282
Noise Level (Error Rate): 0.014501891551071844
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      3084
           1       0.99      0.99      0.99      3260

    accuracy                           0.99      6344
   macro avg       0.99      0.99      0.99      6344
weighted avg       0.99      0.99      0.99      6344

This email is classified as ham (not spam).
Model saved as 'hari_spam_model.pkl'
